# City street network orientations

Author: [Geoff Boeing](https://geoffboeing.com/)

Compare the spatial orientations of city street networks with OSMnx.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

'0.11'

In [2]:
# define the study sites as label : query
places = {'Atlanta'       : 'Atlanta, GA, USA',
         #'Boston'        : 'Boston, MA, USA',
          'Buffalo'       : 'Buffalo, NY, USA',
         #'Charlotte'     : 'Charlotte, NC, USA',
         #'Chicago'       : 'Chicago, IL, USA',
          'Cleveland'     : 'Cleveland, OH, USA',
         #'Dallas'        : 'Dallas, TX, USA',
         #'Houston'       : 'Houston, TX, USA',
          'Denver'        : 'Denver, CO, USA',
         #'Detroit'       : 'Detroit, MI, USA',
          'Las Vegas'     : 'Las Vegas, NV, USA',
         #'Los Angeles'   : {'city':'Los Angeles', 'state':'CA', 'country':'USA'},
         #'Manhattan'     : 'Manhattan, NYC, NY, USA',
          'Miami'         : 'Miami, FL, USA',
          'Minneapolis'   : 'Minneapolis, MN, USA',
         #'Orlando'       : 'Orlando, FL, USA',
         #'Philadelphia'  : 'Philadelphia, PA, USA',
         #'Phoenix'       : 'Phoenix, AZ, USA',
         #'Portland'      : 'Portland, OR, USA',
         #'Sacramento'    : 'Sacramento, CA, USA',
          'San Francisco' : {'city':'San Francisco', 'state':'CA', 'country':'USA'},
         #'Seattle'       : 'Seattle, WA, USA',
         #'St Louis'      : 'St. Louis, MO, USA',
         #'Tampa'         : 'Tampa, FL, USA',
          'Washington'    : 'District of Columbia, USA'}

In [3]:
# verify OSMnx geocodes each query to what you expect (a [multi]polygon geometry)
gdf = ox.gdf_from_places(places.values())
gdf

,geometry,place_name,bbox_north,bbox_south,bbox_east,bbox_west
0,"MULTIPOLYGON (((-84.55107 33.72079, -84.55096 ...","Atlanta, Fulton County, Georgia, United States",33.886823,33.647808,-84.289560,-84.551068
1,"POLYGON ((-78.91945 42.94717, -78.91913 42.946...","Buffalo, Erie County, New York, United States ...",42.966469,42.826039,-78.795168,-78.919453
2,"POLYGON ((-81.87909 41.39641, -81.87906 41.395...","Cleveland, Cuyahoga County, Ohio, United States",41.604436,41.390628,-81.532744,-81.879094
3,"POLYGON ((-105.10988 39.62710, -105.10761 39.6...","Denver, Denver County, Colorado, United States...",39.914209,39.614315,-104.599689,-105.109885
4,"POLYGON ((-115.41463 36.19463, -115.41453 36.1...","Las Vegas, Clark County, Nevada, United States...",36.380491,36.129554,-115.062066,-115.414628
5,"POLYGON ((-80.31976 25.76249, -80.31968 25.762...","Miami, Miami-Dade County, Florida, United Stat...",25.855783,25.709052,-80.139157,-80.319760
6,"POLYGON ((-93.32916 44.94142, -93.32914 44.940...","Minneapolis, Hennepin County, Minnesota, Unite...",45.051250,44.890150,-93.193858,-93.329163
7,"MULTIPOLYGON (((-123.17382 37.77573, -123.1737...","San Francisco, California, United States of Am...",37.929844,37.640314,-122.280016,-123.173825
8,"POLYGON ((-77.11977 38.93428, -77.11900 38.933...","Washington, D.C., United States of America",38.995852,38.791630,-76.909366,-77.119766


## Get the street networks and their edge bearings

In [4]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [5]:
bearings = {}
for place in sorted(places.keys()):
    print(datetime.datetime.now(), place)
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

2019-12-04 11:07:04.150668 Atlanta
2019-12-04 11:07:47.265704 Buffalo
2019-12-04 11:07:57.895716 Cleveland
2019-12-04 11:08:18.155257 Denver
2019-12-04 11:09:10.610305 Las Vegas
2019-12-04 11:09:53.734088 Miami
2019-12-04 11:10:10.721075 Minneapolis
2019-12-04 11:10:32.055285 San Francisco
2019-12-04 11:10:59.697784 Washington


## Visualize it

In [6]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [7]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Century Gothic', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'Century Gothic', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Century Gothic', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [8]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

# add super title and save full image
suptitle_font = {'family':'Century Gothic', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('City Street Network Orientation', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()